# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../files/final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 84 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   match_id                            2280 non-null   object 
 1   goals                               2280 non-null   float64
 2   assists                             2280 non-null   float64
 3   penalties_scored                    2280 non-null   float64
 4   penalties_attempted                 2280 non-null   float64
 5   yellow_cards                        2280 non-null   float64
 6   red_cards                           2280 non-null   float64
 7   expected_goals                      2280 non-null   float64
 8   non_penalty_expected_goals          2280 non-null   float64
 9   expected_assisted_goals             2280 non-null   float64
 10  progressive_carries                 2280 non-null   float64
 11  progressive_passes                  2280 no

In [4]:
combined.head()

,match_id,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,m-00381,0.254701,0.318367,-0.005625,0.002508,-0.047715,-0.033456,0.289988,0.287982,0.280614,...,6,5,9,4,11,9,2,2,0,0
1,m-00382,0.355817,0.442575,-0.047976,-0.073576,-0.129363,-0.034457,0.183910,0.236025,0.276044,...,8,5,16,3,9,14,5,2,0,0
2,m-00383,0.367944,0.472253,0.042021,0.050154,-0.188196,0.008172,0.274595,0.234472,0.335028,...,6,2,1,3,10,14,2,5,0,0
3,m-00384,0.166890,0.417890,-0.009795,-0.001662,0.427224,-0.014343,0.247707,0.247173,0.211108,...,5,1,5,6,13,15,0,2,0,0
4,m-00385,-0.092090,0.391628,-0.163058,-0.195860,-0.306317,-0.013773,-0.250837,-0.094149,0.224704,...,5,7,8,3,15,12,4,1,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed"
]

player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Columns for prediction

### Goals

```
goals
expected_goals
penalties scored/penalties attempted
assists
expected_assists
expected_assisted_goals
key_passes
passes_into_final_third
passes_into_penalty_area
crosses_into_penalty_area
carries_into_final_third
carries_into_penalty_area
shots
shots_on_target
```

### 

## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../files/feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = len(stats_columns)
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-07-16 16:25:03.490024: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


37/37 [==============================] - 0s 406us/step - loss: 34.7381 - accuracy: 0.4013
Epoch 2/500
37/37 [==============================] - 0s 334us/step - loss: 16.4143 - accuracy: 0.4890
Epoch 3/500
37/37 [==============================] - 0s 344us/step - loss: 11.9915 - accuracy: 0.4879
Epoch 4/500
37/37 [==============================] - 0s 324us/step - loss: 9.2030 - accuracy: 0.4890
Epoch 5/500
37/37 [==============================] - 0s 315us/step - loss: 8.7808 - accuracy: 0.5296
Epoch 6/500
37/37 [==============================] - 0s 313us/step - loss: 8.4318 - accuracy: 0.5395
Epoch 7/500
37/37 [==============================] - 0s 338us/step - loss: 8.2636 - accuracy: 0.5422
Epoch 8/500
37/37 [==============================] - 0s 358us/step - loss: 8.1676 - accuracy: 0.5444
Epoch 9/500
37/37 [==============================] - 0s 367us/step - loss: 8.0204 - accuracy: 0.5389
Epoch 10/500
37/37 [==============================] - 0s 318us/step - loss: 8.0716 - accuracy: 0.539

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 449us/step
[ 1.2082345   1.4966671  12.221851   12.721097    4.091545    4.6200285
  5.032965    5.1214066  10.632736   10.361551    1.742518    1.7782214
  0.06187122  0.07524042]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 385us/step - loss: 28.4952 - accuracy: 0.4539
Epoch 2/500
37/37 [==============================] - 0s 527us/step - loss: 14.2864 - accuracy: 0.5428
Epoch 3/500
37/37 [==============================] - 0s 891us/step - loss: 11.0825 - accuracy: 0.5515
Epoch 4/500
37/37 [==============================] - 0s 526us/step - loss: 9.5917 - accuracy: 0.5543
Epoch 5/500
37/37 [==============================] - 0s 362us/step - loss: 8.8756 - accuracy: 0.5559
Epoch 6/500
37/37 [==============================] - 0s 302us/step - loss: 8.3902 - accuracy: 0.5685
Epoch 7/500
37/37 [==============================] - 0s 304us/step - loss: 8.3775 - accuracy: 0.5587
Epoch 8/500
37/37 [==============================] - 0s 315us/step - loss: 8.0739 - accuracy: 0.5614
Epoch 9/500
37/37 [==============================] - 0s 362us/step - loss: 7.9181 - accuracy: 0.5647
Epoch 10/500
37/37 [==============================] - 0s 508us/step - loss: 7.8926 - acc

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 302us/step
[1.29226220e+00 1.86769962e+00 1.03773775e+01 1.44517241e+01
 3.39886379e+00 5.26154518e+00 4.15811634e+00 5.51712370e+00
 1.08381767e+01 1.09394236e+01 2.01410222e+00 1.88222885e+00
 6.67684227e-02 2.06365436e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 497us/step - loss: 25.2790 - accuracy: 0.5027
Epoch 2/500
37/37 [==============================] - 0s 443us/step - loss: 12.1528 - accuracy: 0.5576
Epoch 3/500
37/37 [==============================] - 0s 426us/step - loss: 10.2774 - accuracy: 0.5565
Epoch 4/500
37/37 [==============================] - 0s 529us/step - loss: 9.3512 - accuracy: 0.5636
Epoch 5/500
37/37 [==============================] - 0s 380us/step - loss: 8.8048 - accuracy: 0.5647
Epoch 6/500
37/37 [==============================] - 0s 353us/step - loss: 8.4757 - accuracy: 0.5620
Epoch 7/500
37/37 [==============================] - 0s 324us/step - loss: 8.2390 - accuracy: 0.5614
Epoch 8/500
37/37 [==============================] - 0s 326us/step - loss: 8.1772 - accuracy: 0.5658
Epoch 9/500
37/37 [==============================] - 0s 326us/step - loss: 8.0538 - accuracy: 0.5620
Epoch 10/500
37/37 [==============================] - 0s 324us/step - loss: 7.8797 - acc

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 313us/step
[1.3804781e+00 1.3977308e+00 9.8708944e+00 1.5393120e+01 3.7636685e+00
 5.6878219e+00 2.7958772e+00 5.2300196e+00 8.5501814e+00 9.2659082e+00
 1.5474184e+00 1.4198325e+00 8.6066155e-03 1.7774558e-01]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 379us/step - loss: 24.9510 - accuracy: 0.4945
Epoch 2/500
37/37 [==============================] - 0s 365us/step - loss: 12.1751 - accuracy: 0.5285
Epoch 3/500
37/37 [==============================] - 0s 333us/step - loss: 10.0794 - accuracy: 0.5504
Epoch 4/500
37/37 [==============================] - 0s 332us/step - loss: 9.1138 - accuracy: 0.5444
Epoch 5/500
37/37 [==============================] - 0s 345us/step - loss: 8.5844 - accuracy: 0.5504
Epoch 6/500
37/37 [==============================] - 0s 331us/step - loss: 8.3004 - accuracy: 0.5532
Epoch 7/500
37/37 [==============================] - 0s 332us/step - loss: 8.0511 - accuracy: 0.5570
Epoch 8/500
37/37 [==============================] - 0s 357us/step - loss: 7.8584 - accuracy: 0.5603
Epoch 9/500
37/37 [==============================] - 0s 342us/step - loss: 7.9240 - accuracy: 0.5548
Epoch 10/500
37/37 [==============================] - 0s 323us/step - loss: 7.7253 - acc

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 318us/step
[ 1.1827685e+00  1.8193301e+00  9.7969799e+00  1.4880392e+01
  3.3959546e+00  5.5525584e+00  3.3783867e+00  5.0218043e+00
  9.1884489e+00  1.0515083e+01  1.5862237e+00  1.4142299e+00
 -7.5757280e-03  5.0573796e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
37/37 [==============================] - 0s 536us/step - loss: 23.6480 - accuracy: 0.5132
Epoch 2/500
37/37 [==============================] - 0s 345us/step - loss: 11.9369 - accuracy: 0.5554
Epoch 3/500
37/37 [==============================] - 0s 336us/step - loss: 9.9824 - accuracy: 0.5647
Epoch 4/500
37/37 [==============================] - 0s 336us/step - loss: 9.1082 - accuracy: 0.5663
Epoch 5/500
37/37 [==============================] - 0s 354us/step - loss: 8.5984 - accuracy: 0.5630
Epoch 6/500
37/37 [==============================] - 0s 333us/step - loss: 8.2723 - accuracy: 0.5603
Epoch 7/500
37/37 [==============================] - 0s 321us/step - loss: 8.1007 - accuracy: 0.5674
Epoch 8/500
37/37 [==============================] - 0s 320us/step - loss: 7.8899 - accuracy: 0.5696
Epoch 9/500
37/37 [==============================] - 0s 320us/step - loss: 7.7857 - accuracy: 0.5691
Epoch 10/500
37/37 [==============================] - 0s 321us/step - loss: 7.7685 - accu

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

15/15 [==============================] - 0s 320us/step
[ 1.0138028   2.2089474   8.990231   12.047882    3.0791388   5.1471896
  5.2270303   3.7310667   9.959453    9.48808     1.7732601   1.4148995
  0.11980832 -0.1209271 ]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1182,3,2,13,16,4,4,9,6,11,16,2,2,0,0
669,3,2,10,17,4,6,2,3,10,9,2,1,0,0
1453,2,2,13,13,9,6,3,6,8,13,1,2,0,0
56,1,0,2,25,1,7,1,10,11,8,3,0,0,0
62,1,5,11,13,4,6,5,7,11,19,2,4,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  penalties_scored  penalties_attempted  yellow_cards  \
1044  0.482036  0.339651          0.064913             0.068424      0.863645   
1042 -0.108937 -0.053879          0.049131             0.014054      0.079059   
1817 -0.610690 -0.234151         -0.031666            -0.040085     -0.106861   
1706  0.101150 -0.016399          0.080944             0.094946     -0.428387   
1895  0.154915  0.023660         -0.015995            -0.003973     -0.047530   
...        ...       ...               ...                  ...           ...   
2180 -0.568452 -0.568653         -0.045355            -0.067590     -0.724034   
479   0.351408  0.020493          0.194244             0.225127      0.344879   
561   1.324769  0.918365          0.062550             0.070185     -0.193737   
2151  0.122771  0.145679          0.038617             0.026165      0.181609   
1785  0.308350  0.304689          0.013625             0.023214      0.082184   

      red_cards  expected_g

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../files/prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
# n=15
# print(X_train)
# pca = PCA(n_components = n, random_state=576)
# pca.fit(X)
# feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
# feature_to_pc_map.to_csv("../files/feature_to_15_pcs.csv")

# dump(pca, '../prediction_pca.bin')
# whole_X_pca = pca.transform(whole_x_train)

# pca = PCA(n_components = n, random_state=576)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)


# feature_to_pc_map

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,0.133175,0.135506,0.085078,0.091756,0.115900,0.061184,0.137220,0.136684,0.142287,0.140168,...,0.093743,0.120097,0.051676,0.055952,0.056956,0.098018,0.048813,0.042439,0.039134,0.031892
1,-0.128817,-0.122467,-0.021067,-0.036465,0.133395,0.026838,-0.104691,-0.108695,-0.103041,-0.105708,...,0.183060,0.106816,0.282043,0.280107,0.275421,0.087718,0.267917,0.253169,0.186481,0.122840
2,0.072472,0.077249,-0.027716,-0.027411,-0.191028,-0.097916,0.021108,0.026381,0.027603,0.059161,...,-0.234034,0.041755,0.188073,0.203891,0.208692,0.280411,0.213814,0.196197,0.104143,0.187488
3,-0.127724,-0.002816,-0.472673,-0.479322,-0.026794,-0.132541,-0.150788,-0.098960,0.019306,0.007443,...,0.076124,0.057232,-0.108736,-0.124606,-0.133394,-0.016110,-0.030953,-0.029704,0.049703,-0.112548
4,-0.048446,-0.080226,0.432122,0.373617,-0.027877,0.049384,-0.046512,-0.100431,-0.076660,-0.054839,...,0.008209,0.123092,-0.045958,-0.057498,-0.058096,-0.039325,-0.110777,-0.045765,-0.210283,-0.131179
5,-0.013415,0.025363,-0.041510,0.004905,0.068467,0.573002,-0.079847,-0.087978,0.010300,0.071358,...,-0.109565,-0.017273,0.088126,0.035695,0.006636,-0.059832,0.095033,0.108226,0.407804,-0.520740
6,-0.097378,-0.069628,-0.162149,-0.146098,0.079574,0.496045,-0.107416,-0.097084,-0.073815,0.028175,...,-0.141056,0.099968,-0.017761,-0.032704,-0.039072,-0.131846,0.059072,0.067329,-0.366728,0.548954
7,0.108822,0.093134,-0.020043,0.019215,-0.069716,0.498944,0.137165,0.147209,0.080361,-0.125165,...,0.196654,-0.055650,-0.082287,-0.093364,-0.100974,0.130355,-0.135364,-0.328867,0.319170,0.268878
8,0.124931,0.127934,-0.034190,-0.012964,0.036259,0.207825,0.139279,0.153853,0.131134,-0.090738,...,0.066108,0.271108,-0.064229,-0.027992,-0.010649,-0.120860,0.189138,0.409579,-0.417856,-0.188406


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(INPUT_SIZE, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500
73/73 [==============================] - 0s 356us/step - loss: 38.8025 - accuracy: 0.2780
Epoch 2/500
73/73 [==============================] - 0s 315us/step - loss: 32.8177 - accuracy: 0.2950
Epoch 3/500
73/73 [==============================] - 0s 325us/step - loss: 31.1298 - accuracy: 0.2867
Epoch 4/500
73/73 [==============================] - 0s 298us/step - loss: 12.3412 - accuracy: 0.5055
Epoch 5/500
73/73 [==============================] - 0s 294us/step - loss: 8.5847 - accuracy: 0.5685
Epoch 6/500
73/73 [==============================] - 0s 295us/step - loss: 8.2440 - accuracy: 0.5740
Epoch 7/500
73/73 [==============================] - 0s 294us/step - loss: 8.1363 - accuracy: 0.5729
Epoch 8/500
73/73 [==============================] - 0s 301us/step - loss: 7.9430 - accuracy: 0.5757
Epoch 9/500
73/73 [==============================] - 0s 316us/step - loss: 7.8860 - accuracy: 0.5757
Epoch 10/500
73/73 [==============================] - 0s 300us/step - loss: 7.8059 - ac

In [33]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 395us/step - loss: 8.4452 - accuracy: 0.4934


[8.445172309875488, 0.4934210479259491]

#### Testing

In [34]:
y_hat = model.predict(X_test)

15/15 [==============================] - 0s 643us/step


In [35]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [36]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if home_shots_hat+away_shots_hat == home_shots+away_shots else 0
	total_shots_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots else 0
	total_shots_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if home_shots_on_target_hat+away_shots_on_target_hat == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if home_goals_hat == home_goals and away_goals_hat == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if home_fouls_hat + away_fouls_hat == home_fouls + away_fouls else 0
	total_fouls_over += 1 if home_fouls_hat + away_fouls_hat > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if home_fouls_hat + away_fouls_hat < home_fouls+ away_fouls else 0

	total_corners += 1 if home_corners_hat + away_corners_hat == home_corners + away_corners else 0
	total_corners_over += 1 if home_corners_hat + away_corners_hat > home_corners + away_corners else 0
	total_corners_under += 1 if home_corners_hat + away_corners_hat < home_corners + away_corners else 0

	goals_over += 1 if home_goals_hat + away_goals_hat > home_goals + away_goals else 0
	goals_under += 1 if home_goals_hat + away_goals_hat < home_goals + away_goals else 0

	all_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat < home_corners + away_corners and home_corners_hat + away_corners_hat < home_corners + away_corners and home_goals_hat + away_goals_hat < home_goals + away_goals else 0
	all_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat > home_corners + away_corners and home_goals_hat + away_goals_hat > home_goals + away_goals else 0

In [37]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.0
total_shots_over: 0.5416666666666666
total_shots_under: 0.4583333333333333


total_shots_on_target: 0.0
total_shots_on_target_over: 0.5526315789473685
total_shots_on_target_under: 0.4473684210526316


total_booking_points: 0.0
total_booking_points_over: 0.4758771929824561
total_booking_points_under: 0.5241228070175439


correct_score: 0.0


winner: 0.5021929824561403


total_corners: 0.0
total_corners_over: 0.543859649122807
total_corners_under: 0.45614035087719296


total_fouls: 0.0
total_fouls_over: 0.4451754385964912
total_fouls_under: 0.5548245614035088


goals_over: 0.5350877192982456
goals_under: 0.4649122807017544


all_over: 0.08991228070175439
all_under: 0.07894736842105263


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [38]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(INPUT_SIZE, activation="relu", input_dim=INPUT_SIZE))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation="relu"))

	model.compile(
		optimizer="sgd",
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [39]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = [49]
learn_rate = [1e-2]
dropout = [0.486]
batch_size = [64]
epochs = [500]
n_h_layers = [3]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
X_train.shape

(1824, 64)

In [43]:
param_under_review = "n_h_layers"
value_list = []
for i in range(0, 15):
	searcher = GridSearchCV(estimator=model, n_jobs=-2, param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)
	searchResults = searcher.fit(X_train, y_train)
	value_list.append(searcher.best_params_[param_under_review])

print(f"[INFO] The best {param_under_review} value after 15 iterations is {sum(value_list)/len(value_list)}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-07-16 16:26:00.610615: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 16:26:00.610801: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 16:26:00.611125: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 2/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.293 total time=   5.0s
[CV 1/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.296 total time=   5.0s
[CV 3/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.283 total time=   5.0s
Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-07-16 16:26:15.448146: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 16:26:15.448357: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-16 16:26:15.448765: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 1/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.300 total time=   5.4s
[CV 3/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.297 total time=   5.5s
[CV 2/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.307 total time=   5.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.288 total time=   4.8s
[CV 1/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.289 total time=   4.8s
[CV 3/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.281 total time=   4.8s
Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-07-16 16:26:40.415755: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 2/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.302 total time=   4.8s
[CV 3/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.272 total time=   4.8s
[CV 1/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.301 total time=   4.7s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 3/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.298 total time=   4.9s
[CV 2/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.279 total time=   4.9s
[CV 1/3] END batch_size=64, dropout=0.486, epochs=500, hidden_layer_one=49, learn_rate=0.01, n_h_layers=3;, score=0.289 total time=   4.9s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END batch_size=64, dropout=0.486, epoch

In [44]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.486,
 'epochs': 500,
 'hidden_layer_one': 49,
 'learn_rate': 0.01,
 'n_h_layers': 3}

In [45]:
hidden_layer_one = tuned_model["hidden_layer_one"]
learn_rate = tuned_model["learn_rate"]
dropout = tuned_model["dropout"]
batch_size = tuned_model["batch_size"]
epochs = tuned_model["epochs"]
n_h_layers = tuned_model["n_h_layers"]

model = get_mlp_model(hidden_layer_one=49, learn_rate=learn_rate, dropout=0.486, n_h_layers=3)
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, )

model.save("../files/stats_regression_model.h5")
# pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500
29/29 [==============================] - 0s 547us/step - loss: 31.0662 - accuracy: 0.3015
Epoch 2/500
29/29 [==============================] - 0s 561us/step - loss: 15.1764 - accuracy: 0.4704
Epoch 3/500
29/29 [==============================] - 0s 634us/step - loss: 12.9759 - accuracy: 0.4896
Epoch 4/500
29/29 [==============================] - 0s 480us/step - loss: 12.3021 - accuracy: 0.5016
Epoch 5/500
29/29 [==============================] - 0s 520us/step - loss: 11.7811 - accuracy: 0.5197
Epoch 6/500
29/29 [==============================] - 0s 461us/step - loss: 11.1671 - accuracy: 0.5307
Epoch 7/500
29/29 [==============================] - 0s 424us/step - loss: 10.8812 - accuracy: 0.5389
Epoch 8/500
29/29 [==============================] - 0s 470us/step - loss: 10.5065 - accuracy: 0.5395
Epoch 9/500
29/29 [==============================] - 0s 470us/step - loss: 10.2523 - accuracy: 0.5488
Epoch 10/500
29/29 [==============================] - 0s 431us/step - loss: 10.499

In [46]:
print(len
	  (X_train))

1824


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?